In [1]:
## Import

import requests
import urllib.request
from selenium import webdriver
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
## Error 1

from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen("https://www.coupang.com/")
bs = BeautifulSoup(html , 'lxml')

HTTPError: HTTP Error 403: Forbidden

In [3]:
## Error 2

from selenium import webdriver

# 크롬 드라이버 초기화
def init_driver():
    return webdriver.Chrome("./chromedriver.exe")

driver = init_driver()
url = "https://www.coupang.com/"
driver.get(url)
driver.find_element_by_css_selector("div.header-searchForm input").value

AttributeError: 'WebElement' object has no attribute 'value'

In [4]:
## request + headers / raise_for_status() ?? 

url = "https://www.coupang.com/"
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36"}
res = requests.get(url, headers = headers)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

In [5]:
## 검색어 url

def get_source(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36"}
    res = requests.get(url, headers = headers)
    res.raise_for_status()
    return BeautifulSoup(res.text, "lxml")

search_url = "https://www.coupang.com/np/search?component=&q="
keyword = "우유"
bs = get_source(search_url + keyword)
bs

<!DOCTYPE html>
<!--[if lte IE 7 ]><html class="lt-ie9 lt-ie8" lang="ko-KR"><![endif]--><!--[if IE 8 ]><html class="lt-ie9" lang="ko-KR"><![endif]--><!--[if (gte IE 9)|!(IE)]><!--><html lang="ko-KR"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="zaNrGtrOLMjglkziY2IvmL8dOXyCWHGArDHqFazJQVI" name="google-site-verification"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="//cart.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupang.com" rel="dns-prefetch"/>
<link href="//assets2.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupangcdn.com" rel="dns-prefetch"/>
<link href="//asset1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//private.coupang.com" rel="dns-prefetch"/>
<link href="//img1a.coupangcdn.com" rel="dns-prefetch"/>
<link href="//image1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//thumbnail1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//static.coupangcdn.com" rel="dns-prefetch"/>
<link href="//www.facebook.com"

In [6]:
items = bs.select('li.search-product')

for n,i in enumerate(items):
    
    # 1. 상품 이름
    print(i.select('div.name')[0].text.strip())
    
    # 2. 상품 가격
    print(i.select('div.price-area strong')[0].text)

    # 3. 상품 단위 가격 (명시되지 않는 경우도 있음)
    try: print(i.select('div.price-area span.unit-price')[0].text)
    except : pass   
    
    # 4. 로켓배송/로킷직구/로켓와우 
    try: print(i.select('div.price-area div.delivery span.badge.falcon img')[0].get('alt'))
    except : pass
    
    # 5. 배송 예정일
    print(i.select('div.price-area div.delivery')[0].text)
    
    # 6. 후기 별점(후기수)
    try : print(i.select('div.other-info')[0].text)
    except : pass   
      
    # 7. 적립금
    print(i.select('div.benefit-badges')[0].text)
    
    print("-----------")
    
    if n > 4:
        break

파스퇴르 무항생제 바른목장우유, 1.8L, 1개
7,500
(100ml당 417원) 
 오늘(화) 1/26  도착 보장  
5.0 (7731)
  최대 375원 적립
-----------
매일우유 오리지널 멸균, 200ml, 24팩
11,080
(100ml당 231원) 
로켓직구
 내일(수) 1/27   도착 보장  
4.5 (75663)
  최대 554원 적립
-----------
서울우유 멸균 흰우유, 1000ml, 10팩
18,520
(100ml당 185원) 
로켓직구
 내일(수) 1/27   도착 보장  
4.5 (121797)
  최대 926원 적립
-----------
이담 더 신선한 우유, 900ml, 2개
6,680
(100ml당 371원) 
 오늘(화) 1/26  도착 보장  
5.0 (782)
  최대 334원 적립
-----------
곰곰 저온살균 우유, 2.3L, 1개
4,830
(100ml당 210원) 
 오늘(화) 1/26  도착 보장  
4.5 (833)
  최대 241원 적립
-----------
남양 맛있는우유 GT, 1.8L, 1개
5,100
(100ml당 283원) 
 오늘(화) 1/26  도착 보장  
5.0 (6517)
  최대 255원 적립
-----------


In [9]:
import pandas as pd

css_items = bs.select('li.search-product')
items = []
for n,i in enumerate(css_items):
    
    # 1. 상품 이름
    name = i.select('div.name')[0].text.strip()
    
    # 2. 상품 가격
    price = int(i.select('div.price-area strong')[0].text.replace(",", ""))

    # 3. 상품 단위 가격 (명시되지 않는 경우도 있음)
    unit_price = ""
    try: unit_price = i.select('div.price-area span.unit-price')[0].text
    except : pass   
    
    # 4. 로켓배송/로킷직구/로켓와우 
    delivery_type = ""
    try: delivery_type = i.select('div.price-area div.delivery span.badge.falcon img')[0].get('alt')
    except : pass
    
    # 5. 배송 예정일
    delivery_date = ""
    try: delivery_date = i.select('div.price-area div.delivery')[0].text
    except : pass 
    # 6. 후기 별점(후기수)
    try : rate = i.select('div.other-info')[0].text
    except : pass

    # 7. 적립금
    plus = i.select('div.benefit-badges')[0].text    
    items.append([name,price,unit_price,delivery_type,delivery_date,rate,plus])

df = pd.DataFrame(items)
df.columns = ["Name" , "Price" , "Unit_Price" , "Delivery_type" , "Delivery_Date" , "Rate" , "Plus"]
df.head(5)

,Name,Price,Unit_Price,Delivery_type,Delivery_Date,Rate,Plus
0,"파스퇴르 무항생제 바른목장우유, 1.8L, 1개",7500,(100ml당 417원),,오늘(화) 1/26 도착 보장,5.0 (7731),최대 375원 적립
1,"매일우유 오리지널 멸균, 200ml, 24팩",11080,(100ml당 231원),로켓직구,내일(수) 1/27 도착 보장,4.5 (75663),최대 554원 적립
2,"서울우유 멸균 흰우유, 1000ml, 10팩",18520,(100ml당 185원),로켓직구,내일(수) 1/27 도착 보장,4.5 (121797),최대 926원 적립
3,"이담 더 신선한 우유, 900ml, 2개",6680,(100ml당 371원),,오늘(화) 1/26 도착 보장,5.0 (782),최대 334원 적립
4,"곰곰 저온살균 우유, 2.3L, 1개",4830,(100ml당 210원),,오늘(화) 1/26 도착 보장,4.5 (833),최대 241원 적립


In [10]:
# 크롬 드라이버 초기화 함수
def init_driver():
    return webdriver.Chrome("./chromedriver.exe")

items = []
driver = init_driver()
page_url = "https://www.coupang.com/np/search?q=%EC%9A%B0%EC%9C%A0&channel=user&page="
page = 0

while(1):
    page += 1
    url = page_url + str(page)
    driver.get(url)
    bs = get_source(url)
    css_items = bs.select('li.search-product')
    try : driver.find_element_by_css_selector('a.btn-next')
    except : break
    
    for n,i in enumerate(css_items):

        # 1. 상품 이름
        name = i.select('div.name')[0].text.strip()

        # 2. 상품 가격
        price = int(i.select('div.price-area strong')[0].text.replace(",", ""))

        # 3. 상품 단위 가격 (명시되지 않는 경우도 있음)
        unit_price = ""
        try: unit_price = i.select('div.price-area span.unit-price')[0].text
        except : pass   

        # 4. 로켓배송/로킷직구/로켓와우 
        delivery_type = ""
        try: delivery_type = i.select('div.price-area div.delivery span.badge.falcon img')[0].get('alt')
        except : pass

        # 5. 배송 예정일
        delivery_date = ""
        try: delivery_date = i.select('div.price-area div.delivery')[0].text
        except : pass 
        
        # 6. 후기 별점(후기수)
        try: rate = i.select('div.other-info')[0].text
        except : pass

        # 7. 적립금
        plus = i.select('div.benefit-badges')[0].text

        items.append([name,price,unit_price,delivery_type,delivery_date,rate,plus])

df = pd.DataFrame(items)
df.columns = ["Name" , "Price" , "Unit_Price" , "Delivery_type" , "Delivery_Date" , "Rate" , "Plus"]
df.to_excel("Coupang {} Searching Result.xlsx".format(keyword) , index=False)
df.head(5)

In [11]:
## Re 패키지를 이용해서 키워드가 들어가있는지 확인 예제

ob = re.search("우유" , "릴리푸리 여아용 MILK 상하복")
if ob == None : 
    print("우유가 없어요")


우유가 없어요


In [13]:
items = []
del_items = []

driver = init_driver()
page_url = "https://www.coupang.com/np/search?q=%EC%9A%B0%EC%9C%A0&channel=user&page="
page = 0

while(1):
    page += 1
    url = page_url + str(page)
    driver.get(url)
    bs = get_source(url)
    css_items = bs.select('li.search-product')
    try : driver.find_element_by_css_selector('a.btn-next')
    except : break
    
    for n,i in enumerate(css_items):

        # 1. 상품 이름
        name = i.select('div.name')[0].text.strip()
        # 광고성 / 물건 이름에 키워드가 들어가지 않는 경우는 제외
        ob = re.search(keyword , name)
        if ob == None : 
            del_items.append(name)
            continue

        # 2. 상품 가격
        price = int(i.select('div.price-area strong')[0].text.replace(",", ""))

        # 3. 상품 단위 가격 (명시되지 않는 경우도 있음)
        unit_price = ""
        try: 
            unit_price = i.select('div.price-area span.unit-price')[0].text
            u
        except : pass   

        # 4. 로켓배송/로킷직구/로켓와우 
        delivery_type = ""
        try: delivery_type = i.select('div.price-area div.delivery span.badge.falcon img')[0].get('alt')
        except : pass

        # 5. 배송 예정일
        delivery_date = ""
        try: delivery_date = i.select('div.price-area div.delivery')[0].text
        except : pass 
        
        # 6. 후기 별점(후기수)
        try: rate = i.select('div.other-info')[0].text
        except : pass

        # 7. 적립금
        plus = i.select('div.benefit-badges')[0].text

        items.append([name,price,unit_price,delivery_type,delivery_date,rate,plus])

df = pd.DataFrame(items)
df.columns = ["Name" , "Price" , "Unit_Price" , "Delivery_type" , "Delivery_Date" , "Rate" , "Plus"]
df.to_excel("Coupang {} Searching Result.xlsx".format(keyword) , index=False)

del_df = pd.DataFrame(del_items)
print(del_df)

                                                     0
0              튼튼닷컴 밀크씨슬 [6개월분] - 1000 mg x 180 캡슐, 1개
1                        매일유업 무항생제인증 슬로우밀크, 190ml, 24개
2                                남양유업 초코에몽, 180ml, 24개
3                               폴스 팜하우스 골드 밀크, 1L, 12개
4                      137 Degrees 아몬드 밀크, 180ml, 36개입
..                                                 ...
376  비에스엔 신타 6 엣지 프로틴 파우더 드링크 믹스 단백질 보충제, 1.06kg, 초...
377        마녀의부엌 위치스 블랜딩티 스트로베리 밀크티베이스 540g, 500ml, 1개
378               Cytosport 몬스터 밀크 초콜릿 브라우니, 2436g, 1개
379  비에스엔 신타 6 엣지 프로틴 파우더 드링크 믹스 단백질 보충제, 1.75kg, 바...
380                      위토스 셀렉션 크리스피 라떼 초콜릿, 125g, 3개

[381 rows x 1 columns]


In [15]:

### 전체 코드

## IMPORT
import requests
import urllib.request
from selenium import webdriver
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

# url-request 함수 , html 리턴
def get_source(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36"}
    res = requests.get(url, headers = headers)
    res.raise_for_status()
    return BeautifulSoup(res.text, "lxml")

# Selenium 드라이버 실행 , driver 리턴
def init_driver():
    return webdriver.Chrome("./chromedriver.exe")

# 키워드를 입력받아서 스크래핑 후 xlsx 저장
def search_coupang(keyword):
    items = []
    driver = init_driver()
    page_url = "https://www.coupang.com/np/search?q=%EC%9A%B0%EC%9C%A0&channel=user&page="
    page = 0
    while(1):
        page += 1
        url = page_url + str(page)
        driver.get(url)
        bs = get_source(url)
        css_items = bs.select('li.search-product')
        try : driver.find_element_by_css_selector('a.btn-next')
        except : break
        for n,i in enumerate(css_items):
            # 1. 상품 이름
            name = i.select('div.name')[0].text.strip()
            # 광고성 / 물건 이름에 키워드가 들어가지 않는 경우는 제외
            ob = re.search(keyword , name)
            if ob == None : 
                continue
            # 2. 상품 가격
            price = int(i.select('div.price-area strong')[0].text.replace(",", ""))
            # 3. 상품 단위 가격 (명시되지 않는 경우도 있음)
            unit_price = ""
            try: 
                unit_price = i.select('div.price-area span.unit-price')[0].text
            except : pass   
            # 4. 로켓배송/로킷직구/로켓와우 
            delivery_type = ""
            try: delivery_type = i.select('div.price-area div.delivery span.badge.falcon img')[0].get('alt')
            except : pass
            # 5. 배송 예정일
            delivery_date = ""
            try: delivery_date = i.select('div.price-area div.delivery')[0].text
            except : pass 
            # 6. 후기 별점(후기수)
            try: rate = i.select('div.other-info')[0].text
            except : pass
            # 7. 적립금
            plus = i.select('div.benefit-badges')[0].text
            items.append([name,price,unit_price,delivery_type,delivery_date,rate,plus])
    df = pd.DataFrame(items)
    df.columns = ["Name" , "Price" , "Unit_Price" , "Delivery_type" , "Delivery_Date" , "Rate" , "Plus"]
    df.to_excel("Coupang {} Searching Result.xlsx".format(keyword) , index=False)
    print("Saving Succesfully")


In [16]:
## 키워드 스크래핑

keyword = "우유"
search_coupang(keyword)

Saving Succesfully
